# Transcriptomic profile estimation with various sample size datasets

In [1]:
import numpy as np
import pandas as pd
import scipy.stats as stats
from scipy import io

from sklearn.metrics.pairwise import cosine_similarity

import sys
sys.path.append("../../scripts")
from noiseReductionMethodology import preprocessing, Raman_model
from analysis_pclda import LDA_model
from predictFunc import calcPrediction
from util import returnValues

## Load data

In [2]:
RAMAN = pd.read_csv("../../data/RAMAN_FINGERPRINT.csv")
TRANSCRIPTOME = pd.read_csv("../../data/TRANSCRIPTOME.csv")
RAMAN_PROCESSED = preprocessing(RAMAN)
RAMAN_PROCESSED.head()

0         1         2         3         4         5         6  \
0  0.777427  0.798224  0.788404  0.760493  0.758397  0.730189  0.688064   
1  0.819071  0.838680  0.801314  0.734114  0.704192  0.677671  0.615264   
2  0.784357  0.799093  0.757752  0.689691  0.634893  0.599651  0.571955   
3  0.911177  0.947830  0.915633  0.857712  0.857826  0.800391  0.765022   
4  0.876973  0.864326  0.801489  0.735828  0.716139  0.703013  0.681590   

          7         8         9  ...       590       591       592       593  \
0  0.620933  0.629037  0.658134  ... -1.786467 -1.778755 -1.773075 -1.801913   
1  0.532932  0.525684  0.557399  ... -1.942792 -1.988656 -2.001273 -2.014088   
2  0.523794  0.562298  0.624879  ... -1.856136 -1.828801 -1.792533 -1.794846   
3  0.717855  0.708535  0.691836  ... -1.736215 -1.723609 -1.730770 -1.767129   
4  0.651481  0.649721  0.639855  ... -1.958845 -1.995900 -2.030070 -2.019414   

        594       595       596       597       598  label  
0 -1.817556 -1.802574 -1.790176 -1.800517 -1.835198      0  
1 -2.021082 -2.015529 -1.991117 -1.972676 -1.962001      0  
2 -1.801112 -1.815291 -1.843331 -1.893848 -1.902506      0  
3 -1.791136 -1.787874 -1.769603 -1.755394 -1.783720      0  
4 -2.005512 -1.999898 -2.020077 -2.048122 -2.082585      0  

[5 rows x 600 columns]

In [3]:
GROUP = [RAMAN_PROCESSED[RAMAN_PROCESSED["label"] == i].reset_index(drop=True) for i in range(RAMAN_PROCESSED["label"].max() + 1)]

In [4]:
colorList = ["gray", "#B51700"]
nameList = ["PCA", "NRM"]

In [5]:
numList = np.arange(6, 60, 6)

## Function

In [6]:
def generateDataList(n, GROUP=GROUP):
    RAMAN_dataList = []
    np.random.seed(0)
    for j in range(100):
        INPUTDATA = pd.DataFrame([])
        for OUT in GROUP:
            idxList = list(np.arange(OUT.shape[0]))
            np.random.shuffle(idxList)
            INPUTDATA = pd.concat([INPUTDATA, OUT.iloc[idxList[:n], :]], axis=0)
        INPUTDATA = INPUTDATA.reset_index(drop=True)

        RAMAN_dataList.append(INPUTDATA)
    return RAMAN_dataList

In [7]:
def performTranscriptomicProfileEstimation(dataList, cutPercentages):
    predict_dataList = []
    percent_to_dim_PCA_dataList = []
    percent_NRM_dataList = []

    for RAMAN_DATA in dataList:
        print(".", end="")
        predict_list = []
        percent_to_dim_PCA = []
        percent_NRM = []

        for cutPercent in cutPercentages:
            raman_model = Raman_model(RAMAN_DATA, cutRange=cutPercent, cutMode="percent_fixedDim")
            raman_model.calcTransformation()

            out = []
            for DATA in [raman_model.RAMAN_PCA, raman_model.RAMAN_NRM]:
                lda_model = LDA_model()
                DATA_LDA = lda_model.fit_transform(DATA)

                DATA_LDA = DATA_LDA.groupby("label").mean()
                DATA_LDA["label"] = np.arange(DATA_LDA.shape[0])

                PREDICT = calcPrediction(TRANSCRIPTOME, DATA_LDA, n_components=0, max_iter=50000)

                out.append(PREDICT)

            predict_list.append(out)
            percent_to_dim_PCA.append(raman_model.k_hat)
            percent_NRM.append(raman_model.percent_tilde)

        predict_dataList.append(predict_list)
        percent_to_dim_PCA_dataList.append(percent_to_dim_PCA)
        percent_NRM_dataList.append(percent_NRM)
    print("")
    return predict_dataList, percent_to_dim_PCA_dataList, percent_NRM_dataList

## Perform transcriptomic profile estimation

**Note**:  
The following cell may take a long time to run, as it performs transcriptomic profile estimation across multiple sample sizes and dimensions.  
To save time, the corresponding precomputed results are already included in this repository at:  
 
`results/SUMMARY_PERCENT_fixedDim_SPOMBE_dataSize{n}.csv` (where `{n}` = 6, 12, 18, 24, 30, 36, 42, 48, 54)

In [8]:
cutPercentages = np.arange(80, 100, 0.5)
cutPercentages

array([80. , 80.5, 81. , 81.5, 82. , 82.5, 83. , 83.5, 84. , 84.5, 85. ,
       85.5, 86. , 86.5, 87. , 87.5, 88. , 88.5, 89. , 89.5, 90. , 90.5,
       91. , 91.5, 92. , 92.5, 93. , 93.5, 94. , 94.5, 95. , 95.5, 96. ,
       96.5, 97. , 97.5, 98. , 98.5, 99. , 99.5])

In [9]:
for n in numList:
    print(f"[n = {n:>2}] ===================================")
    dataList = generateDataList(n)
    predict_dataList, percent_to_dim_PCA_dataList, percent_NRM_dataList = performTranscriptomicProfileEstimation(dataList, cutPercentages)
    
    SUMMARY_TABLE = pd.DataFrame(columns=[f"{i:.1f}%" for i in cutPercentages]).T
    SUMMARY_TABLE["dim_PCA"] = np.mean(percent_to_dim_PCA_dataList, axis=0)
    SUMMARY_TABLE["dim_PCA_std"] = np.std(percent_to_dim_PCA_dataList, axis=0)
    SUMMARY_TABLE["percent_NRM"] = np.mean(percent_NRM_dataList, axis=0)
    SUMMARY_TABLE["percent_NRM_std"] = np.std(percent_NRM_dataList, axis=0)
    
    out_pca = np.vstack([np.array([np.sum((returnValues(data[0]) - returnValues(TRANSCRIPTOME)) ** 2, axis=1).sum()
                               for data in predict_list])
                     for predict_list in predict_dataList])

    out_nrm = np.vstack([np.array([np.sum((returnValues(data[1]) - returnValues(TRANSCRIPTOME)) ** 2, axis=1).sum()
                                   for data in predict_list])
                         for predict_list in predict_dataList])

    pList = np.array([stats.wilcoxon(a, b, alternative="two-sided").pvalue for a, b in zip(out_pca.T, out_nrm.T)])

    SUMMARY_TABLE["PRESS_PCA"] = np.mean(out_pca, axis=0)
    SUMMARY_TABLE["PRESS_NRM"] = np.mean(out_nrm, axis=0)
    SUMMARY_TABLE["PRESS_PCA_std"] = np.std(out_pca, axis=0)
    SUMMARY_TABLE["PRESS_NRM_std"] = np.std(out_nrm, axis=0)
    SUMMARY_TABLE["PRESS_diff"] = np.mean(out_nrm - out_pca, axis=0)
    SUMMARY_TABLE["PRESS_diff_std"] = np.std(out_nrm - out_pca, axis=0)
    SUMMARY_TABLE["PRESS_pVal"] = pList

    out_pca_perCon = np.vstack([np.vstack([np.sum((returnValues(predict_list[i][0]) - returnValues(TRANSCRIPTOME)) ** 2, axis=1)
                                           for predict_list in predict_dataList]).mean(axis=0)
                                for i in range(len(cutPercentages))])

    out_nrm_perCon = np.vstack([np.vstack([np.sum((returnValues(predict_list[i][1]) - returnValues(TRANSCRIPTOME)) ** 2, axis=1)
                                           for predict_list in predict_dataList]).mean(axis=0)
                                for i in range(len(cutPercentages))])


    SUMMARY_TABLE.loc[:, [f"PRESS_PCA_c{i + 1}" for i in range(len(GROUP))]] = out_pca_perCon
    SUMMARY_TABLE.loc[:, [f"PRESS_NRM_c{i + 1}" for i in range(len(GROUP))]] = out_nrm_perCon
    
    
    out_pca = np.vstack([np.array([np.hstack([cosine_similarity(a.reshape(1, -1), b.reshape(1, -1))[0]
                                              for a, b in zip(returnValues(data[0]), returnValues(TRANSCRIPTOME))]).mean()
                                   for data in predict_list])
                         for predict_list in predict_dataList])

    out_nrm = np.vstack([np.array([np.hstack([cosine_similarity(a.reshape(1, -1), b.reshape(1, -1))[0]
                                              for a, b in zip(returnValues(data[1]), returnValues(TRANSCRIPTOME))]).mean()
                                   for data in predict_list])
                         for predict_list in predict_dataList])

    pList = np.array([stats.wilcoxon(a, b, alternative="two-sided").pvalue for a, b in zip(out_pca.T, out_nrm.T)])

    SUMMARY_TABLE["cosine_PCA"] = np.mean(out_pca, axis=0)
    SUMMARY_TABLE["cosine_NRM"] = np.mean(out_nrm, axis=0)

    SUMMARY_TABLE["cosine_diff"] = np.mean(out_nrm - out_pca, axis=0)
    SUMMARY_TABLE["cosine_diff_std"] = np.std(out_nrm - out_pca, axis=0)
    SUMMARY_TABLE["cosine_pVal"] = pList

    SUMMARY_TABLE.to_csv(f"../../results/SUMMARY_PERCENT_fixedDim_SPOMBE_dataSize{n}.csv", index=True)
    
    print(f"============================================")

[n =  6] ===================================
....................................................................................................
[n = 12] ===================================
....................................................................................................
[n = 18] ===================================
....................................................................................................
[n = 24] ===================================
....................................................................................................
[n = 30] ===================================
....................................................................................................
[n = 36] ===================================
....................................................................................................
[n = 42] ===================================
...............................................................................